# Analisando os resultados

In [50]:
EXPERIMENTS_PATH = '../TCC/experiments'
EXPERIMENT_NUMBER = 1
import pandas as pd
import numpy as np
import os

### Experimento 1

#### Primeiros passos

In [13]:
YEAR = 2023
df = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/{YEAR}/enem_2023.csv')
df.head()

,id,year,area,sp_area,day,body,alternatives,answer,GPT-3.5-turbo-answers,GEMINI_2023_answers,Llama2_2023_answers,GPT3.5_2023_answers
0,01I,2023,LC - Linguagens e Códigos,Inglês,D1,"QUESTÃO 01 \r\nNo man is an island,\r\nEntire...",\r\nA medo da morte.\r\nB ideia de conexão.\r\...,B,B,B,B,B
1,02I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 02 \r\nThings We Carry on the Sea\r\n...,\r\nA risco de choques culturais.\r\nB impacto...,D,D,D,D,D
2,03I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 03 \r\nDescrição do cartaz: Foto de p...,\r\nA os lixões precisam de ampliação. \r\nB o...,B,B,B,B,B
3,04I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 04 \r\nDescrição do cartum: Em um esc...,\r\nA criticar um padrão de vestimenta.\r\nB d...,B,B,B,C,C
4,05I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 05 \r\nSpanglish (fragmento)\r\nTato ...,\r\nA convergência linguístico-cultural.\r\nB ...,A,A,A,A,A


In [15]:
df['GPT_result'] = df['GPT3.5_2023_answers'] == df['answer']
df['GPT_result'].value_counts()

GPT_result
True     110
False     75
Name: count, dtype: int64

In [14]:
df['GPT_UI_result'] = df['GPT-3.5-turbo-answers'] == df['answer']
df['GPT_UI_result'].value_counts()

GPT_UI_result
True     128
False     57
Name: count, dtype: int64

In [21]:
df['Llama2_result'] = df['Llama2_2023_answers'] == df['answer']
df['Llama2_result'].value_counts()

Llama2_result
False    95
True     90
Name: count, dtype: int64

In [9]:
df['GEMINI_result'] = df['GEMINI_2023_answers'] == df['answer']
df['GEMINI_result'].value_counts()

GEMINI_result
True     132
False     53
Name: count, dtype: int64

In [16]:
similar_answers = df[(df['Llama2_2023_answers'] == df['GPT-3.5-turbo-answers']) & (df['GPT-3.5-turbo-answers'] == df['GEMINI_2023_answers'])]
len(similar_answers)

78

In [19]:
df['answer'].value_counts()

answer
A          46
C          38
D          37
B          34
E          29
Anulado     1
Name: count, dtype: int64

In [18]:
df.Llama2_2023_answers.value_counts()

Llama2_2023_answers
C    66
B    46
D    34
E    27
A    12
Name: count, dtype: int64

In [30]:
display(df.query('Llama2_result == True ')['Llama2_2023_answers'].value_counts())

Llama2_2023_answers
C    27
B    20
D    19
A    12
E    12
Name: count, dtype: int64

#### Análise 2018 - 2023

In [23]:
df_2018 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2018/enem_2018.csv')
display(df_2018.head())
df_2019 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2019/enem_2019.csv')
# display(df_2019.head())
df_2020 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2020/enem_2020.csv')
# display(df_2020.head())   
df_2022 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2022/enem_2022.csv')
# display(df_2022.head())
df_2023 = pd.read_csv(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/2023/enem_2023.csv')
# display(df_2023.head())   

,id,year,area,sp_area,day,body,alternatives,answer,GEMINI_2018_answers,GPT3.5_2018_answers,Llama2_2018_answers
0,01I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 01 \nLava Mae: Creating Showers on Wh...,\nA empregar moradores de rua em lava a jatos ...,B,B,B,B
1,02I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 02 \nDescrição do cartum:\nO cartum a...,\nA se aposente prematuramente.\nB amadureça p...,B,B,B,B
2,03I,2018,LC - Linguagens e Códigos,Inglês,D1,"QUESTÃO 03 \nDon't write in English, they sai...",\nA usa a língua inglesa com efeito humorístic...,D,D,D,D
3,04I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 04 \nTEXTO I\n A Free World-class Edu...,\nA elogiar o trabalho proposto para a educaçã...,E,E,E,E
4,05I,2018,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 05 \n1984 (excerpt)\nGeorge Orwell\n‘...,\nA boicote a ideais libertários.\nB veto ao c...,C,C,C,C


In [26]:
def check_models_answers(df, year):
    print(df[f'GEMINI_{year}_answers'].value_counts())
    print('Soma das respostas do GEMINI:', df[f'GEMINI_{year}_answers'].value_counts().sum())
    print(df[f'GPT3.5_{year}_answers'].value_counts())
    print('Soma das respostas do GPT3.5:', df[f'GPT3.5_{year}_answers'].value_counts().sum()) 
    print(df[f'Llama2_{year}_answers'].value_counts())
    print('Soma das respostas do Llama2:', df[f'Llama2_{year}_answers'].value_counts().sum())

In [48]:
def check_models_results(df, year):
    print(df[f'GEMINI_{year}_answers'].value_counts())
    df['GPT_result'] = df[f'GPT3.5_{year}_answers'] == df['answer']
    print(df['GPT_result'].value_counts())
    df['Llama2_result'] = df[f'Llama2_{year}_answers'] == df['answer']
    print(df['Llama2_result'].value_counts())
    df['GEMINI_result'] = np.where(df[f'GEMINI_{year}_answers'] == 'No answer', 'No answer', df[f'GEMINI_{year}_answers'] == df['answer'])
    print(df['GEMINI_result'].value_counts())


In [54]:
check_models_answers(df_2018, df_2018['year'].unique()[0])
check_models_results(df_2018, df_2018['year'].unique()[0])

GEMINI_2018_answers
B            43
E            38
A            36
C            35
D            30
No answer     3
Name: count, dtype: int64
Soma das respostas do GEMINI: 185
GPT3.5_2018_answers
E    46
B    42
D    41
C    36
A    20
Name: count, dtype: int64
Soma das respostas do GPT3.5: 185
Llama2_2018_answers
C    69
B    53
E    31
D    25
A     7
Name: count, dtype: int64
Soma das respostas do Llama2: 185
GEMINI_2018_answers
B            43
E            38
A            36
C            35
D            30
No answer     3
Name: count, dtype: int64
GPT_result
True     96
False    89
Name: count, dtype: int64
Llama2_result
False    100
True      85
Name: count, dtype: int64
GEMINI_result
True         120
False         62
No answer      3
Name: count, dtype: int64


In [104]:
result_counts = df_2018['GEMINI_result'].value_counts()
# result_counts.columns = ['GEMINI_result', df_2018['year'].unique()[0]]
# print(result_counts)
result_counts_total_df = pd.DataFrame(result_counts).transpose()

display(result_counts_total_df)

GEMINI_result,True,False,No answer
count,120,62,3


### Experimento 2

In [31]:
df = pd.read_csv(f'{EXPERIMENTS_PATH}/{2}/{YEAR}/enem_2023.csv')
df.head()

,id,year,area,sp_area,day,body,alternatives,answer,GPT3.5_2023_answers
0,01I,2023,LC - Linguagens e Códigos,Inglês,D1,"QUESTÃO 01 \nNo man is an island,\nEntire of ...",\nA medo da morte.\nB ideia de conexão.\nC con...,B,B
1,02I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 02 \nThings We Carry on the Sea\nWe c...,\nA risco de choques culturais.\nB impacto do ...,D,D
2,03I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 03 \nDescrição do cartaz: Foto de par...,\nA os lixões precisam de ampliação. \nB o des...,B,B
3,04I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 04 \nDescrição do cartum: Em um escri...,\nA criticar um padrão de vestimenta.\nB desta...,B,D
4,05I,2023,LC - Linguagens e Códigos,Inglês,D1,QUESTÃO 05 \nSpanglish (fragmento)\nTato Lavi...,\nA convergência linguístico-cultural.\nB cara...,A,A


In [32]:
df['GPT_result'] = df['GPT3.5_2023_answers'] == df['answer']
df['GPT_result'].value_counts()

GPT_result
True     113
False     72
Name: count, dtype: int64